## Import packages

In [2]:
from cobra.io.sbml import read_sbml_model
from mewpy.simulation import get_simulator
from mewpy.problems import GOUProblem

### Read model

In [3]:
model = read_sbml_model("yeast-7-GEM_curcumin.xml")

### Analyze the different genes, reactions and metabolites in our model.

In [3]:
model.genes.curs

Gene identifier,curs
Name,
Memory address,0x7f213664f2b0
Functional,True
In 1 reaction(s),curs


In [4]:
model.reactions.curs

Reaction identifier,curs
Name,Curcumin synthase
Memory address,0x7f2136545220
Stoichiometry,s_0803 + s_feruloylCoA + s_feruloyl_diketideCoA --> s_0456 + 2.0 s_0529 + s_curcumin H2O [cytoplasm] + feruloylCoA + feruloyl diketideCoA --> carbon dioxide [cytoplasm] + 2.0 coenzyme A [cytoplasm] + curcumin
GPR,curs
Lower bound,0.0
Upper bound,1000.0


In [5]:
model.metabolites.s_curcumin

Metabolite identifier,s_curcumin
Name,curcumin
Memory address,0x7f21368ade20
Formula,C21H20O6
Compartment,c
In 2 reaction(s),"curs, EX_curcumin"


### Set the different variables


In [4]:
PRODUCT_ID = 'EX_curcumin'
BIOMASS_ID = 'r_4041'
GLC = 'r_1714'
XYL = 'r_1718'
GLY = 'r_1808'
O2 = 'r_1992'
AMO = 'r_1654'
PHO = 'r_2005'

env_conds = [   {GLC: (-10.0, 999999.0)},
                {GLC: (-10.0, 999999.0), O2: (-1, 100000.0)},
                {GLY: (-20.0, 999999.0), GLC: (-10.0, 999999.0), O2: (-1, 100000.0)},
                {GLC: (-10.0, 999999.0), GLY: (-20.0, 999999.0), PHO: (-1.0, 999999.0)},
                {GLC: (-10.0, 999999.0), PHO: (-1.0, 999999.0)},
                {XYL: (-12.0, 999999.0)},
                {XYL: (-12.0, 999999.0), O2: (-1, 100000.0)},
                {GLY: (-20.0, 999999.0), O2: (-1, 100000.0)},
                {GLY: (-20.0, 999999.0)}
                ]

### Solution we want to evaluate with the different associated expression levels. 


In [5]:
solution = {'YPR060C': 8, 'YBR249C': 8, 'YDR127W': 8, 'YGL148W': 8, 'YDR035W': 8, 'YBR166C': 8, 'YLR134W': 0, 'YDR380W': 0}#, 'YNR019W': 0, 'YLR081W': 0, 'YJR009C': 0, 'YGL125W': 32, 'YOR025W': 0, 'YKL085W': 0.0625, 'YBR265W': 0.5, 'YKL141W': 0}

In [6]:
def simul (solution, obj):
    """Function that allows you to simulate a solution using a pFBA

    Args:
        solution (dict): Gene Modifications 
        obj (str or dict): Objective Function

    Returns:
        float: Curcumin and biomass fluxes
    """
    problem = GOUProblem(model=model, env = env_conds[0])
    constraints = problem.solution_to_constraints(solution)
    simul = get_simulator(model, env_conds[0])
    simul.objective = obj
    resul = simul.simulate(method="pFBA",constraints=constraints)
    curcumin_antes = resul.fluxes["EX_curcumin"]
    biomass_antes = resul.fluxes["r_4041"]
    return curcumin_antes, biomass_antes

def simul_with_fix_lowerbound_biomass (solution, obj, biomass_value):
    """Function that allows you to simulate a solution using a pFBA changing the biomass lower bound


    Args:
        solution (dict): Gene Modifications 
        obj (str or dict): Objective Function
        biomass_value (float): Biomass fluxes

    Returns:
        _type_: _description_
    """
    problem = GOUProblem(model=model, env = env_conds[0])
    constraints = problem.solution_to_constraints(solution)
    model.reactions.r_4041.lower_bound = biomass_value
    simul = get_simulator(model, env_conds[0])
    simul.objective = obj
    resul = simul.simulate(method="pFBA",constraints=constraints)
    curcumin_dps = resul.fluxes["EX_curcumin"]
    biomass_dps = resul.fluxes["r_4041"]
    return curcumin_dps, biomass_dps

In [ ]:
b = simul(solution, BIOMASS_ID)
b

In [ ]:
simul_with_fix_lowerbound_biomass(solution, PRODUCT_ID, b[1])

### Small piece of code that allows you to analyze different levels of gene expression


In [ ]:
exp_list = [8, 10, 12, 16, 24, 32]
result = []
model.reactions.r_4041.lower_bound = 0.496

for c in exp_list:
    solution = {'YPR060C': c, 'YBR249C': c, 'YDR127W': c, 'YGL148W': c, 'YDR035W': c, 'YBR166C': c, 'YLR134W': 0, 'YDR380W': 0}
    a = simul(solution, {PRODUCT_ID:0})
    b = simul(solution, {PRODUCT_ID:1})
    print(c, (a,b))

### "FVA" Simulator

In [7]:
print(solution)
env = env_conds[0]
problem = GOUProblem(model=model, env = env)
constraints = problem.solution_to_constraints(solution)
print(constraints)
reaction_sd = model.reactions.r_4041
reaction_sd.lower_bound = 0.496
simul = get_simulator(model,envcond= env)
simul.objective = {PRODUCT_ID:1}   # FVA Upper
# simul.objective = {PRODUCT_ID:0}   # FVA Lower
a = simul.simulate(method="pFBA", constraints=constraints)

{'YPR060C': 8, 'YBR249C': 8, 'YDR127W': 8, 'YGL148W': 8, 'YDR035W': 8, 'YBR166C': 8, 'YLR134W': 0, 'YDR380W': 0}
{'r_0020': (0.20022092809883155, 10000), 'r_0039': (0.20022092809883155, 10000), 'r_0040': (0.20022092809883155, 10000), 'r_0065': (0.20022092809883155, 10000), 'r_0278': (0.17870645833717125, 10000), 'r_0279': (0.20022092809883155, 10000), 'r_0854': (0, 0), 'r_0939': (0.07727027872145598, 10000), 'r_0996': (0.20022092809883155, 10000), 'r_0997': (0.20022092809883155, 10000)}


### Small section where we can get the main fluxes obtained in each solution

In [41]:
print(solution)
print("Objective --> Biomass",simul.objective, end=" || ") 
print("Enviroment --> ",simul.environmental_conditions)
print("Biomass -->", a.fluxes["r_4041"] , end="  ||  ")
print("Curcumin -->",a.fluxes["EX_curcumin"])
print("--------------------Fluxes---------------------------")
print(a.fluxes["r_0851"], "| phenylalanine transaminase")
print(a.fluxes["PAL"], "| PAL")
print(a.fluxes["r_TAL"], "| r_TAL")
print(a.fluxes["hpaBC"], "| hpaBC")
print(a.fluxes["comt"], "| comt")
print(a.fluxes["ferA"], "| ferA")
print(a.fluxes["curs"], "| curs")
print("-------------------------------Reactions---------------------")
print(a.get_metabolite("s_1032"), "     L-phenylalanine cytoplasm") 
print(a.get_metabolite("s_1051"), "     L-tyrosine  cytoplasm") 
print(a.get_metabolite("s_cinnamic"), "     s_cinnamic")
print(a.get_metabolite("s_coumaric"), "     s_coumaric")
print(a.get_metabolite("s_caffeic"), "     s_caffeic")
print(a.get_metabolite("s_ferulic"), "     s_ferulic")
print(a.get_metabolite("s_feruloylCoA"), "     s_feruloylCoA")
print(a.get_metabolite("s_feruloyl_diketideCoA"), "     s_feruloyl_diketideCoA")
print(a.get_metabolite("s_curcumin"), "     s_curcumin")
print(a.get_metabolite("s_0951"))
print(a.get_metabolite("s_1377"), "PPA") #PPA
print(a.get_metabolite("s_0349"),"DAHP") #DAHP   
print(a.get_metabolite("s_0180"),"ARO8") #ARO8   


{'YPR060C': 8, 'YBR249C': 8, 'YDR127W': 8, 'YGL148W': 8, 'YDR035W': 8, 'YBR166C': 8, 'YLR134W': 0, 'YDR380W': 0}
Objective --> Biomass {'EX_curcumin': 1.0} || Enviroment -->  {'r_1714': (-10.0, 999999.0)}
Biomass --> 0.496  ||  Curcumin --> 0.9761550840438842
--------------------Fluxes---------------------------
-0.2939792 | phenylalanine transaminase
0.0 | PAL
1.9523101680877684 | r_TAL
1.9523101680877684 | hpaBC
1.9523101680877684 | comt
1.9523101680877684 | ferA
0.9761550840438842 | curs
-------------------------------Reactions---------------------
             Value
Reaction          
PAL      -0.000000
r_0851    0.293979
r_0852   -0.066414
r_2117   -0.227565
r_2118    0.000000
r_1215    0.000000
r_2004   -0.000000
r_2110   -0.000000      L-phenylalanine cytoplasm
             Value
Reaction          
r_TAL    -1.952310
r_0695   -0.000000
r_1063    2.002902
r_1066   -0.050592
r_2119    0.000000
r_1220    0.000000
r_1221   -0.000000
r_1222    0.000000
r_1223    0.000000
r_2086   -0.

# In this segment, to help visualize the flows for the Lab, Escher was used.

## Escher

In [11]:
import escher
from escher import Builder

In [51]:
# a.fluxes
# fluxes_dict = list(a.fluxes)

In [12]:
reaction_data = a.dataframe
reaction_data

In [39]:
builder = Builder()

# Beauty is the enemy of practicality
builder.model= model
builder.map_json = "Final.json"    #Curcumin Pathways
builder.map_json = "Final_extenso.json"   # Curcumin Pathways + Tal and Pal branches

builder.reaction_data= reaction_data
builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }]
builder.reaction_scale_preset = 'GaBuRd'
builder.highlight_missing = True

builder

Builder(highlight_missing=True, reaction_data=[{'EX_curcumin': 0.9761550840438842, 'curs': 0.9761550840438842,…